# Test data handler

In [24]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [25]:
train_dataset_config = {
'root_dir': "C:\\Users\\thanhdh6\\Documents\\datasets\\menwomen1",         "csv_file_path":'C:\\Users\\thanhdh6\\Documents\\projects\\vinbrain_internship\\image_classifier\\data\\train.csv',
'img_size':224,
'label_cols_list': ['Labels']}

In [26]:
test_dataset_config = {
'root_dir': "C:\\Users\\thanhdh6\\Documents\\datasets\\menwomen1",
'csv_file_path': 'C:\\Users\\thanhdh6\\Documents\\projects\\vinbrain_internship\\image_classifier\\data\\test.csv',
'img_size':224,
'label_cols_list': ['Labels']}

In [27]:
cfgs = {'img_size': 224, 'bs': 2, 'n_workers': 2}

In [28]:
import torchvision.transforms as transforms
transform_train = transforms.Compose([
        transforms.ToPILImage(),
        # transforms.ToTensor(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)),
    ])
transform_test = transforms.Compose([
        transforms.ToPILImage(),
        # transforms.ToTensor(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
    ])

In [29]:
transforms=[transform_train,transform_test]
dataset_configs = (train_dataset_config, test_dataset_config)

In [33]:
from data.men_women_dataset import MenWomenDataset
from data.data_handler import DataHandler
datahandler = DataHandler(ds_class=(MenWomenDataset, MenWomenDataset), transforms=transforms, dataset_configs=dataset_configs, configs=cfgs)

In [34]:
datahandler.show_batch(1,6,mode='train')

AttributeError: 'DataHandler' object has no attribute 'test_ds'

# Test trainer

In [35]:
trainer_configs = {
'model_path': '',
'validate': 0.7,
'lr': 0.001,
'num_epochs': 10,
'steps_save_loss': 100,
'output_folder': 'C:\\Users\\thanhdh6\\Documents\\projects\\vinbrain_internship\\image_classifier\\train\\logs',
'device': 'cpu',
'gpu_id': 0,
# 'batch_size':2,
'lr_schedule':None,
'config_files':'C:\\Users\\thanhdh6\\Documents\\projects\\vinbrain_internship\\image_classifier\\configs\\cifar_configs.py',
'loss_file':"loss_file.txt"
}

In [39]:
from torch.optim import SGD
from torchvision.models import resnet18
from torch import nn

from utils.metric import Accuracy
from models.cnn import CNN, TransferNet

In [40]:
net = TransferNet(model_base=resnet18, pretrain=True,
                    fc_channels=[512], num_classes=2)
optimizer_config={
    'momentum':0.9
}
optimizer = SGD(net.parameters(),lr=1e-4,momentum=0.9)
metric = Accuracy(threshold=0.5, from_logits=True)
crition= nn.CrossEntropyLoss()
lr_schedule = None

In [42]:
from train.trainer import Trainer
trainer = Trainer(net, datahandler, optimizer, crition,transform_test, metric, lr_schedule=None,configs=trainer_configs)
trainer.train()

  0%|          | 0/10 [00:00<?, ?it/s]Epoch 0 step 0
	Loss average 0.014268
	Loss valid average 0.702499, acc valid 0.564565
learning_rate  0.0001
Epoch 0 step 99
	Loss average 0.660466
	Loss valid average 0.538531, acc valid 0.780781
learning_rate  0.0001
Epoch 0 step 198
	Loss average 0.610031
	Loss valid average 0.484747, acc valid 0.729730
learning_rate  0.0001
  0%|          | 0/10 [03:03<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import glob
from skimage import io
list_img_name = glob.glob("E:\data\MenWoman\women\*112.jpg")
list_imgs = []
for img_name in list_img_name:
    img = io.imread(img_name)
    list_imgs.append(img)
output = trainer.get_prediction(list_imgs)

In [5]:
from utils.metric import Precision
precision = Precision(2)
trainer.evaluate("test", precision)

(tensor(0.7536), tensor([0.3882, 0.5092]))

In [7]:
from utils.metric import Precision, Recall, F1_Score, Fbeta_Score
trainer.evaluate("test", Recall(2))

(tensor(0.7536), tensor([0.4521, 0.4439]))

In [8]:
trainer.evaluate("test", F1_Score(2))

(tensor(0.7536), tensor([0.4177, 0.4743]))

In [9]:
trainer.evaluate("test", Fbeta_Score(2))

(tensor(0.7536), tensor([0.2397, 0.2968]))

In [ ]:
from dataset.dataset import ListDataset
import torch
dataset = ListDataset(list_imgs, trainer.transform_test)
dataloader = torch.utils.data.DataLoader(dataset, shuffle= False, batch_size = trainer.batch_size)

In [ ]:
tensor = iter(dataloader).next()
trainer.net(tensor)
# tensor.shape

# Test model

In [ ]:
from model.CNN import TransferNet
import torchvision
from torchvision.models import vgg16


In [ ]:
import torch
net = TransferNet(vgg16, [25088, 4096, 4096], 2, False, True)
image = torch.empty([64, 3, 224, 224])
image = image.to(trainer.device)
output = net(image)
# net

In [ ]:
output = net(tensor)
output.shape

In [ ]:
model_base_list = list(net.children())
# for model in model_base_list:
#     print(model)
#     for param in model.parameters():
#         print(param)
#         print(param.requires_grad)


# Test trainer

In [ ]:
from train.train import Trainer

In [ ]:
import torch
trainer = Trainer(configs, data)
trainer.load_checkpoint("checkpoint_19")
output1, label1 = trainer.get_output("test")

In [ ]:
trainer.metric(output1, label1)

In [ ]:
configs.batch_size = 4
data = CIFARData(configs)
trainer = Trainer(configs, data)
trainer.load_checkpoint("checkpoint_19")
output2, label2 = trainer.get_output("test")
trainer.metric(output2, label2)

In [ ]:
configs.batch_size = 8
data = CIFARData(configs)
trainer = Trainer(configs, data)
trainer.load_checkpoint("checkpoint_19")
output3, label3 = trainer.get_output("test")
trainer.metric(output3, label3)

In [ ]:
from utils.metric import Precision
trainer.evaluate("test", Precision())

In [ ]:
for i in range(len(output1)):
    for j in range(len(output3)):
        if (output1[i] == output3[j]).sum().item() > 0:
            print(i, j)

In [ ]:
data.train_dataset[0][0]

In [ ]:
from utils.metric import Accuracy
trainer.evaluate("test", Accuracy())

In [ ]:
configs.batch_size = 32
trainer = Trainer(configs, data)
trainer.load_checkpoint("checkpoint_19")
trainer.evaluate("test")

# # Test metric

In [ ]:
from utils.metric import Accuracy
import torch

In [ ]:
class Accuracy:
    def __init__(self, threshold = 0.5, from_logits:bool = True):
        self.threshold = threshold
        self.from_logis = from_logits
    
    def __call__(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        if self.from_logis:
            y_pred, y_true = self._conversion(y_pred, y_true, self.threshold)
        return torch.mean((y_pred == y_true).float())
    
    def _conversion(self, y_pred, y_true, threshold):
        if len(y_pred.shape) == len(y_true.shape) + 1:
            y_pred = torch.argmax(y_pred, dim = 1)
        if len(y_pred.shape) == len(y_true.shape) and y_pred.dtype == torch.float:
            y_pred = (y_pred > threshold).float()
        return y_pred, y_true

In [ ]:
import torch
torch.clip

In [ ]:
class Precision:
    def __init__(self, epsilon = 1e-10):
        self.epsilon = epsilon

    def __call__(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        true_positive = torch.sum(torch.round(torch.clamp(y_pred*y_true, 0, 1)))
        num_positive = torch.sum(torch.round(torch.clamp(y_pred, 0, 1)))
        return true_positive/(num_positive+self.epsilon)

In [ ]:
from utils.metric import Fbeta_Score
import torch
x = torch.Tensor([1, 0, 1, 1])
# y = torch.zeros((4,), dtype = torch.int64)
y = torch.Tensor([0, 1, 0, 1])

betaScore = Fbeta_Score()
betaScore(x, y)

In [ ]:
from utils.metric import IOU

import torch
x = torch.tensor([[
    [1, 0, 0],
    [1, 0, 0],
    [1, 0, 0],
]])
y = torch.tensor([[
    [1, 0, 0],
    [1, 1, 0],
    [1, 0, 0],
]])
iou = IOU(0.5)
iou(x, y)


# Split train test

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

from dataset.MenWomanDataset import split_train_test_folder
split_train_test_folder('E:\data\MenWoman', 0.9)

In [ ]:
import os
def load_train_img(file_path):
    print(file_path)
    file_ = open(file_path, "r")
    list_ = file_.readlines()
    list_img = []
    list_label = []
    for line in list_:
        img_name, label = line.replace("\n","").split(",")
        list_img.append(img_name)
        list_label.append(int(label))
    return list_img, list_label
list_img, list_label = load_train_img("E:\data\data\\test.txt")

from skimage import io
io.imread(os.path.join("E:\data\data", list_img[0]))

In [ ]:
from skimage import io


In [ ]:
import numpy as np
import torch

class Accuracy:
    def __init__(self, threshold = 0.5, from_logits:bool = True):
        self.threshold = threshold
        self.from_logis = from_logits
    
    def __call__(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        if self.from_logis:
            y_pred, y_true = self._conversion(y_pred, y_true, self.threshold)
        return torch.mean((y_pred == y_true).float())
    
    def _conversion(self, y_pred, y_true, threshold):
        if len(y_pred.shape) == len(y_true.shape) + 1:
            y_pred = torch.argmax(y_pred, dim = 1)
        if len(y_pred.shape) == len(y_true.shape) and y_pred.dtype == torch.float:
            y_pred = (y_pred > threshold).float()
        return y_pred, y_true
acc2 = Accuracy()
trainer.metric = acc2